In [ ]:
import os
import sys
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

SCENENN_DIR = "/external_datasets/sceneNN"

In [ ]:
# import sceneNN
from dataset import *

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Validation dataset
dataset_test = ObjectsDataset()
dataset_test.load(SCENENN_DIR, "testing", skip=59)
dataset_test.prepare()

## Detection

In [ ]:
class InferenceConfig(ObjectsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_test.image_ids)
# image_id = 328
original_image, image_meta, class_ids, bbox, mask=\
    modellib.load_image_gt(dataset_test, inference_config, 
                           image_id, use_mini_mask=False)
print(dataset_test.image_info[image_id]['path'])
image = original_image[:, :, 0:3]
depth = original_image[:, :, 3]
print("image_id", image_id)
log("image", image)
log("depth", depth)
log("image_meta", image_meta)
log("bbox", bbox)
log("mask", mask)

depth_norm = 1.0 - depth / np.max(depth)
rgbd = np.dstack((
    image[:, :, 0] * depth_norm,
    image[:, :, 1] * depth_norm,
    image[:, :, 2] * depth_norm))
rgbd = rgbd.astype(np.uint8)
depth = np.dstack((depth_norm * 255,) * 3)
depth = depth.astype(np.uint8)

ax = get_ax()
ax.axis('off')
ax.imshow(depth)

ax = get_ax()
ax.axis('off')
ax.imshow(rgbd)

visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names, limit=1)

visualize.display_instances(image, bbox, mask, class_ids, dataset_test.class_names, figsize=(8, 8))
# visualize.display_instances(depth, bbox, mask, class_ids, dataset_test.class_names, figsize=(8, 8))

In [ ]:
# model_paths = model.find_all()
# for model_path in model_paths:
#     print("Loading weights from ", model_path[0])
#     for checkpoint in model_path[1]:
#         print("    " + checkpoint)
        # model.load_weights(checkpoint, by_name=True)
model.load_weights(model.find_last()[1], by_name=True)
# MODEL_PATH = os.path.join(ROOT_DIR, "logs/seg_scenenn20171109T1726/mask_rcnn_seg_scenenn_0115.h5")
model.load_weights(MODEL_PATH, by_name=True)
start = time.clock()
r = model.detect([original_image])[0]
print(time.clock() - start)
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                    dataset_test.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
%%script false
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_test.image_ids, 50)
APs = []
for i, image_id in enumerate(image_ids):
    # Load image and ground truth data
    image_path = dataset_test.image_info[image_id]['path']
    try:
        image, image_meta, gt_class_id, gt_bbox, gt_mask  =\
            modellib.load_image_gt(dataset_test, inference_config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
        APs.append(AP)
        print('{}: {}: {}: {}'.format(i + 1, image_id, image_path, AP))
    except:
        print("Error processing image {}".format(image_path))

print("mAP: ", np.mean(APs))